In [3]:
import queue
import sounddevice as sd
import numpy as np

audio_queue = queue.Queue()
sample_rate = 16000
frame_duration = 0.02  # 20 ms
frame_size = int(sample_rate * frame_duration)

def callback(indata, frames, time, status):
    mono_data = indata.copy()
    audio_queue.put(mono_data)

# Open the microphone stream
with sd.InputStream(channels=1, samplerate=sample_rate, blocksize=frame_size, callback=callback):
    print("Recording...")
    try:
        while True:
            frame = audio_queue.get()
    except KeyboardInterrupt:
        print("Stopped recording.")
 

Recording...
Stopped recording.


In [4]:
from pyannote.audio.pipelines import VoiceActivityDetection
import torch

INFO:speechbrain.utils.quirks:Applied quirks (see `speechbrain.utils.quirks`): [disable_jit_profiling, allow_tf32]
INFO:speechbrain.utils.quirks:Excluded quirks specified by the `SB_DISABLE_QUIRKS` environment (comma-separated list): []


In [ ]:
pipeline = VoiceActivityDetection(segmentation="pyannote/segmentation", use_auth_token="TOKEN")

Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.5.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint C:\Users\Максим\.cache\torch\pyannote\models--pyannote--segmentation\snapshots\660b9e20307a2b0cdb400d0f80aadc04a701fc54\pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.3.2. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.5.1+cpu. Bad things might happen unless you revert torch to 1.x.


In [ ]:
import numpy as np

accumulator = np.array([], dtype=np.float32)
accumulation_window = 1.0  # 1 second
samples_per_window = int(sample_rate * accumulation_window)

with sd.InputStream(device=1, channels=1, samplerate=sample_rate, blocksize=frame_size, callback=callback):
    print("Recording... Press Ctrl+C to stop.")
    try:
        while True:
            frame = audio_queue.get()
            accumulator = np.concatenate((accumulator, frame.flatten()))
            if len(accumulator) >= samples_per_window:
                chunk = accumulator[:samples_per_window]
                accumulator = accumulator[samples_per_window:]
                waveform = torch.tensor(chunk, dtype=torch.float32).unsqueeze(0)
                vad_result = pipeline({"waveform": waveform, "sample_rate": sample_rate})
                print(vad_result)
    except KeyboardInterrupt:
        print("Stopped recording.")


Recording... Press Ctrl+C to stop.

[ 00:00:00.351 -->  00:00:01.026] A SPEECH

[ 00:00:00.638 -->  00:00:01.026] A SPEECH
[ 00:00:00.047 -->  00:00:00.570] A SPEECH
[ 00:00:00.419 -->  00:00:01.026] A SPEECH






[ 00:00:00.132 -->  00:00:00.773] A SPEECH

[ 00:00:00.030 -->  00:00:01.026] A SPEECH
[ 00:00:00.638 -->  00:00:01.026] A SPEECH
[ 00:00:00.030 -->  00:00:00.368] A SPEECH





[ 00:00:00.030 -->  00:00:01.026] A SPEECH
[ 00:00:00.030 -->  00:00:01.026] A SPEECH











































































































































































































































































































































































































































































































































In [ ]:
import sounddevice as sd
print(sd.query_devices())